# EDA BIOMETRICO

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import locale
import holidays

In [2]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [3]:
ruta ="dataset/produccion10.xls"
df_cota = pd.read_excel(ruta)
año = 2025
mes = 10

WARNING *** file size (1953747) not 512 + multiple of sector size (512)


### BD info produccion

In [4]:
ruta2 ="empleados/info producciòn.xlsx"
df_empleados = pd.read_excel(ruta2)

In [5]:
df_cota.isnull().sum()

Tiempo                  0
ID de Usuario         829
Nombre                829
Apellido              976
Número de tarjeta    5520
Dispositivo             0
Punto del evento        0
Verificación            0
Estado                  0
Evento                  0
Notas                5654
dtype: int64

### LIMPIEZA

In [6]:
# Manejar por buenas practicas titulos en minuscula y sin espacios en blanco
new_name =[] 
for column in df_cota.columns:
    minus = column.lower()
    replace = minus.replace(" ","_")
    new_name.append(replace)

df_cota.columns = new_name


df_cota.duplicated().sum()


np.int64(0)

In [7]:
#cambia el nombre de la columna de numero a ID del trabajador
df_cota.rename(columns={'número': 'id'}, inplace=True) 

df_cota["nombre_completo"] = df_cota["nombre"] +" "+df_cota["apellido"]

### MATCH TRABAJADORES BIOMETRICO

In [8]:
from fuzzywuzzy import process, fuzz
import pandas as pd
import unidecode

# ---- Función de normalización ----
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    s = str(nombre).upper().strip()
    s = unidecode.unidecode(s)
    s = " ".join(s.split())
    return s

# ---- Normalizar columnas ----
df_cota['nombre_limpio']     = df_cota['nombre_completo'].apply(limpiar_nombre)
df_empleados['empleado_limpio'] = df_empleados['Empleado'].apply(limpiar_nombre)

# ---- Listas únicas útiles (sin cadenas vacías) ----
cotas_unique = [n for n in df_cota['nombre_limpio'].unique() if n and len(n) >= 3]
# Usamos el df_empleados tal cual para mantener Departamento y nombre original
THRESHOLD = 85  # ajustar si quieres más/menos estricto

# ---- Buscamos, por cada empleado oficial, su mejor match en df_cota ----
# Guardamos el mejor match por nombre_cota (para evitar duplicados incorrectos)
best_map = {}   # key = nombre_limpio (df_cota), value = dict(oficial, dept, score, empleado_limpio_oficial)

for _, row in df_empleados.iterrows():
    oficial_limpio = row['empleado_limpio']
    if not oficial_limpio:
        continue
    dept = row['Departamento']
    # buscar mejor match en cotas_unique usando token_set_ratio
    resultado = process.extractOne(oficial_limpio, cotas_unique, scorer=fuzz.token_set_ratio)
    if resultado:
        match_name, score = resultado[0], resultado[1]  # extractOne con lista devuelve (match, score)
        # solo consideramos si pasa el umbral
        if score >= THRESHOLD:
            prev = best_map.get(match_name)
            # si ya hay un candidato para ese nombre, quedarnos con el de mayor score
            if (prev is None) or (score > prev['score']):
                best_map[match_name] = {
                    'oficial': row['Empleado'],         # nombre oficial legible
                    'empleado_limpio_oficial': oficial_limpio,
                    'dept': dept,
                    'score': score
                }

# ---- Construir mapas y aplicarlos al df_cota ----
map_depto   = {k: v['dept']     for k, v in best_map.items()}
map_oficial = {k: v['oficial']  for k, v in best_map.items()}

df_cota['Departamento']    = df_cota['nombre_limpio'].map(map_depto)
df_cota['empleado_matched'] = df_cota['nombre_limpio'].map(map_oficial)

# ---- Informes rápidos para diagnosticar ----
print("Oficiales (total):", df_empleados.shape[0])
print("Nombres únicos en df_cota:", len(cotas_unique))
print("Nombres de df_cota asignados a oficiales:", len(map_depto))
print("Umbral usado (THRESHOLD):", THRESHOLD)

empleados_matcheados = sorted({v['empleado_limpio_oficial'] for v in best_map.values()})
empleados_sin_match = df_empleados[
    ~df_empleados['empleado_limpio'].isin(empleados_matcheados)
]['Empleado'].tolist()

print("Empleados oficiales sin match:", len(empleados_sin_match))
print(empleados_sin_match)




/home/donsson/proyectos/env/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Oficiales (total): 37
Nombres únicos en df_cota: 57
Nombres de df_cota asignados a oficiales: 36
Umbral usado (THRESHOLD): 85
Empleados oficiales sin match: 1
['ESTEFANY ALVAREZ VILLANUEVA']


In [9]:
print(f"El numero de trabajadores unicos es de ",df_cota['nombre_limpio'].nunique())

El numero de trabajadores unicos es de  58


In [10]:
df_cota_produccion = df_cota[df_cota["Departamento"].isin(["DIRECCIÓN DE PLANTA","PRODUCCION","MANTENIMIENTO"])]

In [11]:
print(f"Numero de trabajadores unicos en planta",df_cota_produccion['nombre_limpio'].nunique())

Numero de trabajadores unicos en planta 36


### HORAS IDEALES DEL MES

In [12]:
import pandas as pd
import holidays

def horas_ideales_a_corte(year, month, fecha_fin_corte):
    """
    Calcula el total de horas ideales de trabajo desde el inicio del mes 
    hasta una fecha de corte específica, excluyendo festivos colombianos.

    :param year: Año del mes a calcular (entero).
    :param month: Mes a calcular (entero).
    :param fecha_fin_corte: Fecha límite para el cálculo (pd.Timestamp o cadena 'YYYY-MM-DD').
    :return: Total de horas ideales redondeado a 2 decimales (flotante).
    """
    # Festivos de Colombia
    # Se genera para el año del inicio del mes y el año de la fecha de corte
    co_holidays = holidays.CountryHoliday("CO", years=[year, fecha_fin_corte.year]) 

    # Rango de días: desde el inicio del mes hasta la fecha de corte
    inicio = pd.Timestamp(year, month, 1)
    # Nos aseguramos que la fecha de corte no sea anterior al inicio del mes
    fin = min(fecha_fin_corte, inicio + pd.offsets.MonthEnd(0)) 
    
    # Asegurarse de que el inicio no sea posterior al fin (en caso de que el mes sea futuro a la fecha de corte)
    if inicio > fin:
        return 0.0

    dias = pd.date_range(inicio, fin, freq="D")

    total_horas = 0.0
    for d in dias:
        if d.date() not in co_holidays:
            if d.weekday() <= 3:  # Lunes a Jueves
                total_horas += 9.5
            elif d.weekday() == 4: # Viernes
                total_horas += 8.5
        # Sábados, domingos y festivos = 0

    return round(total_horas, 2)

# --- EJEMPLO DE USO ---

# Definir los parámetros para el cálculo
año = año
mes = mes
# NOTA: Debes ingresar la fecha de corte como un objeto Timestamp de pandas.
# Si el día de corte es, por ejemplo, el 15 de agosto de 2025:
fecha_corte = pd.Timestamp(año, mes, 27) # Puedes cambiar el día (ej: 15)

# Cálculo
horas_del_mes_a_corte = horas_ideales_a_corte(año, mes, fecha_corte)

# Resultado
print(F"\n📅 Calculando Horas Ideales a Corte para el mes {mes}/{año}")
print(F"✂️ Fecha de corte: **{fecha_corte.strftime('%Y-%m-%d')}**")
print(F"🕒 Total de horas ideales esperadas a corte: **{horas_del_mes_a_corte}**")



📅 Calculando Horas Ideales a Corte para el mes 10/2025
✂️ Fecha de corte: **2025-10-27**
🕒 Total de horas ideales esperadas a corte: **167.0**


In [13]:
import pandas as pd
import numpy as np
from datetime import time, date, timedelta
import holidays

# --- Funciones Auxiliares ---

def _is_holiday(d: date, holidays_obj) -> bool:
    """Verifica si una fecha es festivo."""
    if holidays_obj is None:
        return False
    try:
        return d in holidays_obj
    except Exception:
        return d in set(holidays_obj)

def horas_ideales_mes(year=2025, month=8, holidays_obj=None, treat_holidays_as_non_working=True):
    """Calcula las horas ideales para un mes completo."""
    inicio = pd.Timestamp(year, month, 1)
    fin = inicio + pd.offsets.MonthEnd(0)
    dias = pd.date_range(inicio, fin, freq="D").date

    total = 0.0
    for d in dias:
        if d.weekday() <= 3:     # Lun-Jue
            jornada = 9.5
        elif d.weekday() == 4:   # Viernes
            jornada = 8.5
        else:
            jornada = 0.0

        if treat_holidays_as_non_working and _is_holiday(d, holidays_obj) and jornada > 0:
            continue
        total += jornada

    return round(total, 2)

# --- Función Principal Modificada ---

def horas_trabajadas_diario_y_mensual(df, year=2025, month=8,
                                     restar_almuerzo=False, minutos_almuerzo=30,
                                     holidays_obj=None, treat_holidays_as_non_working=True,
                                     debug=False):
    dfx = df.copy()

    if not pd.api.types.is_datetime64_any_dtype(dfx['tiempo']):
        dfx['tiempo'] = pd.to_datetime(dfx['tiempo'], errors='coerce')

    dfx = dfx[dfx['tiempo'].notna()].copy()
    dfx = dfx[(dfx['tiempo'].dt.year == year) & (dfx['tiempo'].dt.month == month)]

    dfx['fecha'] = dfx['tiempo'].dt.date

    agg = (
        dfx.groupby(['nombre_limpio', 'fecha'], as_index=False)
        .agg(
            primera=('tiempo', 'min'),
            ultima=('tiempo', 'max'),
            marcaciones=('tiempo', 'count')
        )
    )

    dept_first = (
        dfx.sort_values(['nombre_limpio', 'fecha', 'tiempo'])
        .groupby(['nombre_limpio', 'fecha'], as_index=False)['Departamento']
        .first()
    )
    agg = agg.merge(dept_first, on=['nombre_limpio', 'fecha'], how='left')

    span_horas = (agg['ultima'] - agg['primera']).dt.total_seconds() / 3600.0
    span_horas = span_horas.where(agg['marcaciones'] >= 2, 0.0)

    if restar_almuerzo:
        span_horas = np.maximum(0.0, span_horas - (minutos_almuerzo / 60.0))

    agg['horas_trabajadas'] = span_horas

    hora_inicio = time(6, 40)
    agg['llegada'] = np.where(
        agg['primera'].isna(),
        "Sin marcación",
        np.where(agg['primera'].dt.time > hora_inicio, "Tarde", "A tiempo")
    )
    
    # 1. CÁLCULO DE HORAS ESPERADAS Y HORAS EXTRA DIARIAS
    
    # Obtener el día de la semana (Lunes=0, Domingo=6)
    agg['weekday'] = pd.to_datetime(agg['fecha']).dt.weekday
    
    # Definir la lógica de horas esperadas por día
    condiciones = [
        agg['weekday'] <= 3,  # Lunes a Jueves: 9.5 horas
        agg['weekday'] == 4   # Viernes: 8.5 horas
    ]
    valores_esperados = [
        9.5,
        8.5
    ]
    
    # Asignar horas esperadas (0.0 para fines de semana por defecto)
    agg['horas_esperadas_dia'] = np.select(condiciones, valores_esperados, default=0.0)
    
    # Aplicar la lógica de festivos
    if treat_holidays_as_non_working and holidays_obj is not None:
        # Identificar días que son festivos Y que NO son sábado/domingo (horas esperadas > 0)
        is_holiday_working_day = agg['fecha'].apply(lambda d: _is_holiday(d, holidays_obj)) & (agg['horas_esperadas_dia'] > 0)
        # Establecer las horas esperadas a 0.0 para esos días
        agg.loc[is_holiday_working_day, 'horas_esperadas_dia'] = 0.0
    
    # 2. CALCULAR HORAS EXTRA
    agg['horas_extra'] = (agg['horas_trabajadas'] - agg['horas_esperadas_dia']).round(2)


    # Diario (reporte con nuevas columnas)
    diario = agg.rename(columns={'nombre_limpio': 'empleado', 'fecha': 'fecha_dia'})[
        ['empleado', 'Departamento', 'fecha_dia', 'marcaciones', 'primera', 'ultima',
         'horas_trabajadas', 'horas_esperadas_dia', 'horas_extra', 'llegada']
    ].copy()

    # Resumen mensual (sin cambios en esta parte, usa la lógica original)
    mensual = (
        diario.groupby(['empleado', 'Departamento'], as_index=False)
        .agg(
            horas_trabajadas_total=('horas_trabajadas', 'sum'),
            horas_extra_total=('horas_extra', 'sum'), # Puedes agregar el total de horas extra aquí
            dias_con_registros=('fecha_dia', 'nunique')
        )
    )
    mensual['horas_trabajadas_total'] = mensual['horas_trabajadas_total'].round(2)
    mensual['horas_extra_total'] = mensual['horas_extra_total'].round(2)

    horas_ideales_const = horas_ideales_mes(year, month, holidays_obj=holidays_obj,
                                             treat_holidays_as_non_working=treat_holidays_as_non_working)
    mensual['horas_ideales'] = horas_ideales_const

    mensual['cumplimiento'] = np.where(
        mensual['horas_trabajadas_total'] >= mensual['horas_ideales'], "Sí", "No"
    )

    mensual = mensual[['empleado', 'Departamento', 'horas_trabajadas_total',
                       'horas_ideales', 'horas_extra_total', 'cumplimiento', 'dias_con_registros']].sort_values('empleado')

    return diario, mensual, horas_ideales_const




In [14]:
diario, mensual, horas_ideales_mes = horas_trabajadas_diario_y_mensual(
    df_cota_produccion,
    year=año,
    month=mes,
    holidays_obj=co_holidays,           #festivos 🇨🇴
    treat_holidays_as_non_working=True, # True = los festivos no suman horas
    restar_almuerzo=False              # ponlo True si se quiere el tiempo real de trabajo que por ahora no
)


In [15]:
import xlsxwriter

year = año
month = mes

nombre_archivo = f"entregas/mes_produccion_{year}_{month:02d}.xlsx"
with pd.ExcelWriter(nombre_archivo, engine="xlsxwriter") as writer:
    # Guardar las dos hojas
    diario.to_excel(writer, sheet_name="Diario", index=False)
    mensual.to_excel(writer, sheet_name="Mensual", index=False)

    # === AQUI EMPIEZA EL FORMATO CONDICIONAL ===
    workbook = writer.book
    ws_mensual = writer.sheets["Mensual"]


    # Columnas clave
    col_cumplimiento = mensual.columns.get_loc("cumplimiento") + 1  # +1 por índice Excel
    col_dias = mensual.columns.get_loc("dias_con_registros") + 1

    ultima_fila = len(mensual) + 1  # +1 por el encabezado

In [16]:
diario[diario["Departamento"]=="MANTENIMIENTO"]

,empleado,Departamento,fecha_dia,marcaciones,primera,ultima,horas_trabajadas,horas_esperadas_dia,horas_extra,llegada
282,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-14,1,2025-10-14 16:13:35,2025-10-14 16:13:35,0.000000,9.5,-9.50,Tarde
283,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-15,2,2025-10-15 06:16:04,2025-10-15 16:11:57,9.931389,9.5,0.43,A tiempo
284,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-16,2,2025-10-16 06:17:55,2025-10-16 16:16:09,9.970556,9.5,0.47,A tiempo
285,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-17,2,2025-10-17 06:12:53,2025-10-17 15:15:29,9.043333,8.5,0.54,A tiempo
286,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-20,2,2025-10-20 06:08:50,2025-10-20 16:12:47,10.065833,9.5,0.57,A tiempo
287,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-21,2,2025-10-21 06:13:48,2025-10-21 16:18:55,10.085278,9.5,0.59,A tiempo
288,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-22,2,2025-10-22 06:07:57,2025-10-22 16:15:56,10.133056,9.5,0.63,A tiempo
289,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-23,2,2025-10-23 06:05:17,2025-10-23 16:11:50,10.109167,9.5,0.61,A tiempo
290,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-24,2,2025-10-24 06:35:27,2025-10-24 15:15:01,8.659444,8.5,0.16,A tiempo
291,ELKIN FABIAN DAZA PLAZAS,MANTENIMIENTO,2025-10-25,2,2025-10-25 06:50:57,2025-10-25 12:02:32,5.193056,0.0,5.19,Tarde
